In [1]:
# check how many events and what type of events are lost from sim to trigger to L1 to L2
import icecube
from icecube import icetray, dataio
from I3Tray import I3Tray
import glob
import os
from datetime import datetime
import json
import csv
import matplotlib.pyplot as plt
import numpy as np

<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for std::vector<char, std::allocator<char> > already registered; second conversion method ignored.
<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for std::vector<std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >, std::allocator<std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > > > already registered; second conversion method ignored.
<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for std::vector<OMKey, std::allocator<OMKey> > already registered; second conversion method ignored.
<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for std::vector<short, std::allocator<short> > already registered; second conversion method ignored.
<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for std::vector<int, std::allocator<int> > already registered; second conversion method ignore

In [5]:
class save_ID(icetray.I3Module):
    def __init__(self, ctx):
        icetray.I3Module.__init__(self,ctx)
        self.AddParameter("event_id_list", "list_of_ids", None)

    def Configure(self):
        self.event_id_list = self.GetParameter("event_id_list")
        
    def DAQ(self, frame):
        self.event_id_list.append( (frame["I3EventHeader"].run_id, frame["I3EventHeader"].event_id) )

class remove_with_ID(icetray.I3Module):
    def __init__(self, ctx):
        icetray.I3Module.__init__(self,ctx)
        self.AddParameter("event_id_list", "list_of_ids", None)

    def Configure(self):
        self.event_id_list = self.GetParameter("event_id_list")
        
    def DAQ(self, frame):
        current_tuple = (frame["I3EventHeader"].run_id, frame["I3EventHeader"].event_id)
        if current_tuple not in self.event_id_list:
            self.PushFrame(frame)
        
    def Physics(self, frame):
        current_tuple = (frame["I3EventHeader"].run_id, frame["I3EventHeader"].event_id)
        if current_tuple not in self.event_id_list:
            self.PushFrame(frame)

In [3]:

path_trigger = "20904_examples/IC86.2020_corsika.020904.000001.i3.zst"
paths = ["first_trial/", "second_trial/", "third_trial/"]
for i, path in enumerate(paths):
    # which L2 events survived?
    event_id_list_L2 = []
    tray = I3Tray()
    tray.Add("I3Reader", filename=path+"L2_sanity_trial"+str(i+1)+".i3.gz")
    tray.Add(save_ID, event_id_list = event_id_list_L2)
    tray.Execute()
    with open(path+"IDs_L2_trial"+str(i+1)+".csv", "w") as f:
        csv_writer = csv.writer(f)
        csv_writer.writerows(event_id_list_L2)
    
    # which trigger did not survive to L2?
    tray = I3Tray()
    tray.Add("I3Reader", filename=path_trigger)
    tray.Add(remove_with_ID, event_id_list = event_id_list_L2) # pass previous list
    tray.Add("I3Writer", filename=path+"trigger_didnt_survive_L2_trial"+str(i+1)+".i3.gz")
    tray.Execute()

NOTICE (I3Tray): I3Tray finishing... (I3Tray.cxx:526 in void I3Tray::Execute(bool, unsigned int))
NOTICE (I3Tray): I3Tray finishing... (I3Tray.cxx:526 in void I3Tray::Execute(bool, unsigned int))
NOTICE (I3Tray): I3Tray finishing... (I3Tray.cxx:526 in void I3Tray::Execute(bool, unsigned int))
NOTICE (I3Tray): I3Tray finishing... (I3Tray.cxx:526 in void I3Tray::Execute(bool, unsigned int))
NOTICE (I3Tray): I3Tray finishing... (I3Tray.cxx:526 in void I3Tray::Execute(bool, unsigned int))
NOTICE (I3Tray): I3Tray finishing... (I3Tray.cxx:526 in void I3Tray::Execute(bool, unsigned int))


In [3]:
# for the CORSIKA already done Level 2

path         = "20904_examples/"
path_trigger = path + "IC86.2020_corsika.020904.000001.i3.zst"
path_L2      = path + "Level2_IC86.2020_corsika.020904.000001.i3.zst"

# which L2 events survived?
event_id_list_L2 = []
tray = I3Tray()
tray.Add("I3Reader", filename=path_L2)
tray.Add(save_ID, event_id_list = event_id_list_L2)
tray.Execute()
with open(path+"IDs_L2_trial0.csv", "w") as f:
    csv_writer = csv.writer(f)
    csv_writer.writerows(event_id_list_L2)

# which trigger did not survive to L2?
tray = I3Tray()
tray.Add("I3Reader", filename=path_trigger)
tray.Add(remove_with_ID, event_id_list = event_id_list_L2) # pass previous list
tray.Add("I3Writer", filename=path+"trigger_didnt_survive_L2_trial0.i3.gz")
tray.Execute()

NOTICE (I3Tray): I3Tray finishing... (I3Tray.cxx:526 in void I3Tray::Execute(bool, unsigned int))
NOTICE (I3Tray): I3Tray finishing... (I3Tray.cxx:526 in void I3Tray::Execute(bool, unsigned int))


### L1 no L2

In [8]:
# first clean L1 files

def clean_L1(frame):
    if frame["PassedAnyFilter"]:
        return True
    else:
        return False

paths = ["first_trial/", "second_trial/", "third_trial/"]
for i, path in enumerate(paths):
    event_id_list_L1 = []
    tray = I3Tray()
    tray.Add("I3Reader", filename=path+"L1_sanity_trial"+str(i+1)+".i3.gz")
    tray.Add(clean_L1, streams=[icetray.I3Frame.DAQ, icetray.I3Frame.Physics])
    tray.Add(save_ID, event_id_list = event_id_list_L1)
    tray.Add("I3Writer", filename=path+"L1_cleaned_trial"+str(i+1)+".i3.gz")
    tray.Execute()
    with open(path+"IDs_L1_trial"+str(i+1)+".csv", "w") as f:
        csv_writer = csv.writer(f)
        csv_writer.writerows(event_id_list_L1)

NOTICE (I3Tray): I3Tray finishing... (I3Tray.cxx:526 in void I3Tray::Execute(bool, unsigned int))
NOTICE (I3Tray): I3Tray finishing... (I3Tray.cxx:526 in void I3Tray::Execute(bool, unsigned int))
NOTICE (I3Tray): I3Tray finishing... (I3Tray.cxx:526 in void I3Tray::Execute(bool, unsigned int))


In [7]:
# L1 but didnt survive L2
paths = ["first_trial/", "second_trial/", "third_trial/"]
for i, path in enumerate(paths):
    # which L2 events survived?
    id_list_string = csv.reader(open(path+"IDs_L2_trial"+str(i+1)+".csv")) # reads as strings
    event_id_list_L2 = [(int(row[0]), int(row[1])) for row in id_list_string] # convert strings to int
    
    # which trigger did not survive to L2?
    tray = I3Tray()
    tray.Add("I3Reader", filename=path + "L1_cleaned_trial"+str(i+1)+".i3.gz")
    tray.Add(remove_with_ID, event_id_list = event_id_list_L2) # pass previous list
    tray.Add("I3Writer", filename=path+"L1_didnt_survive_L2_trial"+str(i+1)+".i3.gz")
    tray.Execute()

NOTICE (I3Tray): I3Tray finishing... (I3Tray.cxx:526 in void I3Tray::Execute(bool, unsigned int))
NOTICE (I3Tray): I3Tray finishing... (I3Tray.cxx:526 in void I3Tray::Execute(bool, unsigned int))
NOTICE (I3Tray): I3Tray finishing... (I3Tray.cxx:526 in void I3Tray::Execute(bool, unsigned int))
